# Correctness bugs testing




In [2]:
import networkx as nx
from networkx.algorithms import bipartite
from sklearn.cluster import KMeans
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from IPython import display

In [5]:
# Premade graphs
g_kite = nx.krackhardt_kite_graph()
g_florentine = nx.florentine_families_graph()
g_karate = nx.karate_club_graph()

# Other example graphs available on the "data" folder of this repo
df_consprot = pd.read_csv('data/PROT_edges.csv', sep = ",")
g_consprot = nx.from_pandas_edgelist(df_consprot, "Source", "Target", create_using=nx.DiGraph) #setting up the directed graph
# subgraph of consprot
node_degree_dict=nx.degree(g_consprot)
g_consprot2=nx.subgraph(g_consprot,[x for x in g_consprot.nodes() if node_degree_dict[x]>=25])

df_trains = pd.read_csv('data/trainnetworknl.csv', names=['origin', 'destination'])
g_trains = nx.Graph()
[g_trains.add_node(node) for node in df_trains.origin]
[g_trains.add_edge(node,edge) for node, edge in zip(df_trains.origin, df_trains.destination)]

df_reviews = pd.read_csv("data/ignatieff_reviews.csv")

df_docs = pd.read_csv('data/docsimilarity.csv', index_col="Unnamed: 0")

g_docs = nx.Graph()
[g_docs.add_node(node) for node in df_docs.source]
[g_docs.add_edge(node,edge, weight = weight) for node, edge, weight in zip(df_docs.source, df_docs.target, df_docs.weight)]

df_furniture = pd.read_csv('data/furniturewebsites.csv')
df_furniture['links'] = df_furniture.links.str.split(', ')
df_furniture = df_furniture.explode('links')
g_furniture = nx.DiGraph()
g_furniture.add_nodes_from(df_furniture.webpage)
g_furniture.add_edges_from(zip(df_furniture.webpage, df_furniture.links))

df_treaties = pd.read_csv('data/mytreaties.csv')
g_treaties = nx.Graph()
[g_treaties.add_node(node, bipartite=0) for node in df_treaties.State if node != np.nan]
[g_treaties.add_node(node, bipartite=1) for node in df_treaties.treaty if node != np.nan]
[g_treaties.add_edge(state, treaty) for state, treaty in zip(df_treaties.State, df_treaties.Affiliation) if treaty != np.nan];

df_history = pd.read_csv("data/legal_history.csv", sep=";")
df_history.set_index("Unnamed: 0", inplace=True)
g_history = nx.from_pandas_adjacency(df_history)
g_history = nx.to_directed(g_history)

# Styling and reproducibility
sns.set_style('whitegrid')
pd.set_option("display.precision", 3)
np.random.seed(123)

# Helper functions

This will compute print some results side by side and return the absolute value of the differences between the official results and the handmade results.

In [54]:
def compare(graph, official, myfunction, head=5):
    officialres = official(graph)
    nodeid = list(dict(officialres).keys())
    officialscore = list(dict(officialres).values())
    myscore = myfunction(graph)
    df = pd.DataFrame({'nodeid':nodeid,
                       'officialscore':officialscore,
                       'myscore':myscore})
    delta = np.abs(officialscore - myscore)
    return print(df.head(head), f"\n **** \n sum of delta for {graph} \n", np.sum(delta))

# 1. Eigenvector Centrality

In [ ]:
def my_eigc(graph, iterations=100):
    A = nx.adjacency_matrix(graph)
    A = A.todense()
    A_dims = A.shape[0]
    v = (np.ones(A_dims)/A_dims)
    for i in range(0,iterations):
        v = A@v.reshape(A_dims,-1)
        mynorm = np.linalg.norm(v)
        res = v / mynorm
    return np.ravel(res)
    
    
    
    
    
    

In [55]:
for i in [g_kite, g_karate, g_trains]:
    compare(i, nx.eigenvector_centrality, my_eigc)

   nodeid  officialscore  myscore
0       0          0.352    0.352
1       1          0.352    0.352
2       2          0.286    0.286
3       3          0.481    0.481
4       4          0.286    0.286 
 **** 
 sum of delta for Graph named 'Krackhardt Kite Social Network' with 10 nodes and 18 edges 
 5.823439517384579e-06
   nodeid  officialscore  myscore
0       0          0.355    0.312
1       1          0.266    0.302
2       2          0.317    0.361
3       3          0.211    0.199
4       4          0.076    0.056 
 **** 
 sum of delta for Graph named "Zachary's Karate Club" with 34 nodes and 78 edges 
 0.9481901566644553
      nodeid  officialscore  myscore
0  Amsterdam          0.390    0.390
1    Utrecht          0.569    0.569
2      Gouda          0.162    0.162
3      Weert          0.058    0.058
4  Eindhoven          0.254    0.254 
 **** 
 sum of delta for Graph with 20 nodes and 29 edges 
 2.1474244097801493e-05


C:\Users\gustavo.arosemena\AppData\Local\Temp\ipykernel_20468\2260359830.py:2: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph)
C:\Users\gustavo.arosemena\AppData\Local\Temp\ipykernel_20468\2260359830.py:2: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph)
C:\Users\gustavo.arosemena\AppData\Local\Temp\ipykernel_20468\2260359830.py:2: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(graph)
